In [1]:
%display latex
x, y, t = var(['x', 'y', 't'])
assume(x, 'real')
assume(y, 'real')
assume(t, 'real')

omega = 2 * pi

visc, gamma, gas_constant, kappa, Pr = var('mu', 'gamma', 'R', 'kappa', 'Pr')

assume(gamma, 'real')
assume(gas_constant, 'real')
assume(visc, 'real')
assume(Pr, 'real')
assume(gas_constant, 'real')


assume(t>=0,visc>=0, kappa>0,gas_constant>0)
assume(gamma > 1)

# Convergence
rho_b = 1
p_b = 1/gamma

rho_0 = Rational('1/2')
p_0 = Rational('1/10')
k = Rational('1/5') * pi * Matrix([1,1])
v_0 = vector([1,1,]) * Rational('1/4')
X = vector([x,y])

In [2]:
def trigrat(expr):
    return SR(maxima(expr).trigrat())

def trigsimp(expr):
    return SR(maxima(expr).trigrat())
    
val = (k * X)[0] - omega * t
val = val.factor()

rho = rho_b + rho_0 * cos(val.factor()) 
v = simplify(v_0 * sin(val))

j = rho * v

p = p_b + p_0 * sin(val)
E = simplify(p/(gamma-1) + (1/2) * (1/rho) * (j * j))

Q = Matrix([rho, j[0], j[1], E])[0]
Q


(1/2*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 1, 1/8*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y), 1/8*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y), 1/32*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2 + 1/10*(10/gamma + sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y))/(gamma - 1))

In [3]:
gradQ = jacobian(Q, X)
gradQ

[                                                                                                                                                                                    -1/10*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)                                                                                                                                                                                     -1/10*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)]
[                                                                                             1/40*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/40*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2                                                                                              1/40*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/40*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2]
[                                                                                             1/40*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/40*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2                                                                                              1/40*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/40*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2]
[1/80*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/160*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^3 + 1/50*pi*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)/(gamma - 1) 1/80*pi*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/160*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^3 + 1/50*pi*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)/(gamma - 1)]

In [4]:
v_div = v[0].diff(x) + v[1].diff(y)
v_grad = jacobian(v, X)
stress_T = visc * (Rational('2/3') * identity_matrix(2) * v_div - (v_grad + v_grad.T))
stress_T

[-1/30*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) -1/10*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)]
[-1/10*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) -1/30*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)]

In [5]:
# Temperature
T = (p/(rho * gas_constant))
flux_heat = (jacobian(T, X)[0])
flux_heat = vector([trigsimp(flux_heat[0]), trigsimp(flux_heat[1])])
flux_heat

(2/25*(2*pi*gamma*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + pi*gamma + 10*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y))/(8*R*gamma*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + R*gamma*cos(-4*pi*t + 2/5*pi*x + 2/5*pi*y) + 9*R*gamma), 2/25*(2*pi*gamma*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + pi*gamma + 10*pi*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y))/(8*R*gamma*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + R*gamma*cos(-4*pi*t + 2/5*pi*x + 2/5*pi*y) + 9*R*gamma))

In [6]:
flux_rho = v * rho
v_outer = v.column() * v.row()
flux_v = rho * v_outer + identity_matrix(2) * p + stress_T
flux_E = v * (identity_matrix(2) * E + identity_matrix(2) * p + stress_T) - kappa * flux_heat

In [7]:
flux = (Matrix([flux_rho, flux_v[0], flux_v[1], flux_E]))
flux

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               1/8*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                1/8*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)]
[                                                                                                                                                                                                                                                                                                                                                                                                                       -1/30*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 1/32*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2 + 1/gamma + 1/10*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)                                                                                                                                                                                                                                                                                                                                                                                                                                                                            -1/10*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 1/32*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2]
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                           -1/10*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 1/32*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2                                                                                                                                                                                                                                                                                                                                                                                                                        -1/30*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 1/32*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2 + 1/gamma + 1/10*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)]
[-1/40*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 1/1920*(16*pi*mu*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 15*(cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + 2)*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y)^2 - 48*(10/gamma + sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y))/(gamma - 1) - 480/gamma - 48*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y))*sin(-2*pi*t + 1/5*pi*x + 1/5*pi*y) - 2/25*(2*pi*gamma*cos(-2*pi*t + 1/5*pi*x + 1/5*pi*y) + pi*gamma + 10*pi*si

In [8]:
source = (jacobian(Q, t) + jacobian(flux[:,0], x) + jacobian(flux[:,1], y))[:,0]

In [9]:
import sympy
from sympy import sympify
from sympy.utilities.codegen import codegen
def sympify_vector(vec):
    components = [sympify(e) for e in vec]
    return sympy.Matrix(components)

gradQSympy = sympify_vector(vector(gradQ.list())).reshape(gradQ.nrows(),gradQ.ncols())

[(c_name, c_code), (h_name, c_header)] = codegen([['evaluateSource', sympify_vector(vector(source.list()))],
                                                  ['evaluateQ', sympify_vector(Q)],
                                                  # gradQ has shape (dim, vars) in exahype!
                                                  ['evaluateGradQ', gradQSympy.T] 
                                                 ],
                                                project='NavierStokes_ManufacturedSolution',
                                                language='C',
                                                standard='C99')
print(c_code)

/******************************************************************************
 *                      Code generated with sympy 1.1.1                       *
 *                                                                            *
 *              See http://www.sympy.org/ for more information.               *
 *                                                                            *
 *          This file is part of 'NavierStokes_ManufacturedSolution'          *
 ******************************************************************************/
#include "evaluateSource.h"
#include <math.h>

void evaluateSource(double R, double gamma, double kappa, double mu, double t, double x, double y, double *out_5175751922508384391) {

   out_5175751922508384391[0] = (1.0L/20.0L)*M_PI*(cos(-2*M_PI*t + (1.0L/5.0L)*M_PI*x + (1.0L/5.0L)*M_PI*y) + 2)*cos(-2*M_PI*t + (1.0L/5.0L)*M_PI*x + (1.0L/5.0L)*M_PI*y) - 1.0L/20.0L*M_PI*pow(sin(-2*M_PI*t + (1.0L/5.0L)*M_PI*x + (1.0L/5.0L)*M_PI*y), 2) + M_